# Artificial Neural Network

### Importing the libraries

In [423]:
import pandas as pd
import numpy as np
import tensorflow as tf

## Part 1 - Data Preprocessing

### Importing the dataset

In [424]:
dataset = pd.read_csv('Churn_Modelling.csv')
X = dataset.iloc[:, 3:-1].values
y = dataset.iloc[:, -1].values

In [425]:
print(X)

[[619 'France' 'Female' ... 1 1 101348.88]
 [608 'Spain' 'Female' ... 0 1 112542.58]
 [502 'France' 'Female' ... 1 0 113931.57]
 ...
 [709 'France' 'Female' ... 0 1 42085.58]
 [772 'Germany' 'Male' ... 1 0 92888.52]
 [792 'France' 'Female' ... 1 0 38190.78]]


In [426]:
print(y)

[1 0 1 ... 1 1 0]


### Encoding categorical data

#### Label Encoding the "Gender" column

In [427]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
X[:, 2] = le.fit_transform(X[:, 2])

In [428]:
print(X)

[[619 'France' 0 ... 1 1 101348.88]
 [608 'Spain' 0 ... 0 1 112542.58]
 [502 'France' 0 ... 1 0 113931.57]
 ...
 [709 'France' 0 ... 0 1 42085.58]
 [772 'Germany' 1 ... 1 0 92888.52]
 [792 'France' 0 ... 1 0 38190.78]]


#### One Hot Encoding the "Geography" column

In [429]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
ct = ColumnTransformer(transformers=[('encoder', OneHotEncoder(), [1])], remainder='passthrough')
X = np.array(ct.fit_transform(X))

In [430]:
print(X[0])

[1.0 0.0 0.0 619 0 42 2 0.0 1 1 1 101348.88]


### Splitting the dataset into the Training set and Test set

In [431]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)


In [432]:
print(X_train[0])

[0.0 0.0 1.0 667 0 34 5 0.0 2 1 0 163830.64]


### Feature Scaling Necessary for deep learning to all features

In [433]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [434]:
print(X_train[0])

[-1.01460667 -0.5698444   1.74309049  0.16958176 -1.09168714 -0.46460796
  0.00666099 -1.21571749  0.8095029   0.64259497 -1.03227043  1.10643166]


## Part 2 - Building the ANN

### Initializing the ANN

In [435]:
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Dense
ann = Sequential()

### Adding the first hidden layer

In [436]:
ann.add(Dense(units=6, activation='relu'))

### Adding the second hidden layer

In [437]:
ann.add(Dense(units=6, activation='relu'))

### Adding the output layer

In [438]:
ann.add(Dense(units=1, activation='sigmoid')) #unit = dimension of output (number of neurons)
#activation in more than 2 categories is softmax

## Part 3 - Training the ANN

### Compiling the ANN

In [439]:
import tensorflow.python.keras.optimizers
ann.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy']) #for non binary classification categorical_crossentropy

### Training the ANN on the Training set

In [440]:
ann.fit(X_train, y_train, batch_size=32, epochs=100)

Epoch 1/100
250/250 [==============================] - 1s 965us/step - loss: 0.5822 - accuracy: 0.7390
Epoch 2/100
250/250 [==============================] - 0s 1ms/step - loss: 0.4672 - accuracy: 0.7966

## Part 4 - Making the predictions and evaluating the model

### Predicting the result of a single observation

**Homework**

Use our ANN model to predict if the customer with the following informations will leave the bank: 

Geography: France

Credit Score: 600

Gender: Male

Age: 40 years old

Tenure: 3 years

Balance: \$ 60000

Number of Products: 2

Does this customer have a credit card ? Yes

Is this customer an Active Member: Yes

Estimated Salary: \$ 50000

So, should we say goodbye to that customer ?

**Solution**

In [441]:
customer = [1, 0, 0, 600, 1, 40, 3, 60000, 2, 1, 1, 50000]
prediction = ann.predict(sc.transform([customer])) > 0.5 #Propability of customer leaving bank
print(prediction)

[[False]]


Therefore, our ANN model predicts that this customer stays in the bank!

**Important note 1:** Notice that the values of the features were all input in a double pair of square brackets. That's because the "predict" method always expects a 2D array as the format of its inputs. And putting our values into a double pair of square brackets makes the input exactly a 2D array.

**Important note 2:** Notice also that the "France" country was not input as a string in the last column but as "1, 0, 0" in the first three columns. That's because of course the predict method expects the one-hot-encoded values of the state, and as we see in the first row of the matrix of features X, "France" was encoded as "1, 0, 0". And be careful to include these values in the first three columns, because the dummy variables are always created in the first columns.

### Predicting the Test set results

In [442]:
y_pred = ann.predict(X_test)
y_pred = (y_pred > 0.5)
print(np.concatenate((y_pred.reshape(len(y_pred),1), y_test.reshape(len(y_test),1)),1))

[[0 0]
 [0 1]
 [0 0]
 ...
 [0 0]
 [0 0]
 [0 0]]


### Making the Confusion Matrix

In [443]:
from sklearn.metrics import confusion_matrix, accuracy_score
cm = confusion_matrix(y_test, y_pred)
print(cm)
accuracy_score(y_test, y_pred)

[[1515   80]
 [ 200  205]]


0.86